## 1. Data Preprocessing

In [1]:
import pandas as pd
import torch
import nltk
import numpy as np

from torch import nn
from tqdm.notebook import tqdm

from utils.helpers import Utility
from utils.model import RNN

# nltk.download('stopwords')

Using TensorFlow backend.


In [2]:
train_result = Utility.generate_tensors_from_csv_file('tn_reviews', None)

In [3]:
tokenizer = train_result['tokenizer']

In [4]:
valid_result = Utility.generate_tensors_from_csv_file('vd_reviews', tokenizer)

In [5]:
test_result = Utility.generate_tensors_from_csv_file('tt_reviews', tokenizer)

In [6]:
test_result.keys()

dict_keys(['tokenizer', 'review_tensor', 'label_tensor'])

## 2. Batch Data

In [7]:
batch_size = 50

In [8]:
train_dataloader = Utility.generate_dataloader_from_tensors(train_result, batch_size)
valid_dataloader = Utility.generate_dataloader_from_tensors(valid_result, batch_size)
test_dataloader = Utility.generate_dataloader_from_tensors(test_result, batch_size)

In [9]:
reviews, labels = next(iter(train_dataloader))

reviews.shape, labels.shape

(torch.Size([50, 5000]), torch.Size([50, 1]))

## 3. Build the Neural Network

In [10]:
train_on_gpu = torch.cuda.is_available()

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):

    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
        
    hidden = tuple([each.data for each in hidden])
    
    # zero accumulated gradients
    rnn.zero_grad()
    
    # get the output from the model
    output, hidden = rnn(inp, hidden)
    
    # calculate the loss and perform backprop
    loss = criterion(output.to(dtype=torch.float), target.to(dtype=torch.float))
    
    loss.backward()
     
    optimizer.step()
    
    return loss.item(), hidden

In [12]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    
    batch_losses = []
    
    
    ###################
    # train the model #
    ###################
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(tqdm(train_dataloader)):
            
            inputs, labels = inputs.to(torch.int64), labels.to(torch.int64)
            
#             # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
#             # record loss
            batch_losses.append(loss)

            
        print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
            epoch_i, n_epochs, np.average(batch_losses)))
        batch_losses = []

    # returns a trained rnn
    return rnn

In [13]:
# Data params
# Sequence Length
# sequence_length = 20  # of words in a sequence
# Batch Size

# data loader - do not change
# train_loader = batch_data(int_text, sequence_length, batch_size)

# Training parameters
# Number of Epochs
num_epochs = 500
# Learning Rate
learning_rate = 0.06

vocab_to_int = tokenizer.word_index

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int) + 1
# Output size
output_size = 1
# Embedding Dimension
embedding_dim = 100
# Hidden Dimension
hidden_dim = 200
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

In [ ]:
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
torch.save(trained_rnn, './output/trained_rnn_final.pt')
print('Model Trained and Saved')

Training for 500 epoch(s)...



Epoch:    1/500   Loss: 0.9763311599142789




Epoch:    2/500   Loss: 0.8226359724355836




Epoch:    3/500   Loss: 0.8741705642556244




Epoch:    4/500   Loss: 0.9189963914313406




Epoch:    5/500   Loss: 0.8783711909926484




Epoch:    6/500   Loss: 0.8645995654507146




Epoch:    7/500   Loss: 0.8560380937275539

